In [1]:
from dotenv import load_dotenv
import os
from langchain.chat_models import AzureChatOpenAI
import warnings
warnings.filterwarnings("ignore")

In [2]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model = AzureChatOpenAI(temperature=0, deployment_name="chat")

# Cadenas de utilidad
* Son funciones que ya tienne un proposito muy especifico; generar un resumen a partir de textos, resolver preguntas, crear una conversación con memoria o sin memoria.


In [10]:
# load_summarize_chain
from langchain.chains.summarize import load_summarize_chain
from langchain_community.chat_models import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
# Cargamos el pdf
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()


chain = load_summarize_chain(model, chain_type="map_reduce", verbose=False)  #stuff o map_reduce

chain.run(docs)

'The article examines the development of autonomous agents using large language models (LLMs). It covers the different aspects of these agents, such as planning, memory, and tool use. The article presents case studies and examples of LLM-powered agents in different fields, including science and simulation. It also addresses the challenges and limitations associated with using LLMs in autonomous agents.'

In [7]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

prompt_template = """Write a concise summary of the following:
"{text}"
CONCISE SUMMARY in spanish:"""
prompt = PromptTemplate.from_template(prompt_template)
llm_chain = LLMChain(llm=model, prompt=prompt)
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")
print(stuff_chain.run(docs))

El artículo discute la idea de construir agentes autónomos utilizando modelos de lenguaje grandes (LLM) como controladores principales. Se presentan varios ejemplos de demostraciones de concepto, como AutoGPT y GPT-Engineer, que demuestran el potencial de los LLM para resolver problemas complejos. El sistema de agente autónomo consta de tres componentes principales: planificación, memoria y uso de herramientas. La planificación implica descomponer tareas complejas en subobjetivos más pequeños y mejorar los resultados a través de la autorreflexión. La memoria incluye la memoria a corto plazo y la memoria a largo plazo, que permite al agente retener y recuperar información. El uso de herramientas implica que el agente aprenda a utilizar API externas para obtener información adicional. Sin embargo, existen desafíos en cuanto a la longitud limitada del contexto, la planificación a largo plazo y la confiabilidad de la interfaz de lenguaje natural.


In [14]:
# Pendiente a ver si funciona
# from langchain.chains import (
#     StuffDocumentsChain,
#     LLMChain,
#     ReduceDocumentsChain,
#     MapReduceDocumentsChain,
# )
# from langchain_core.prompts import PromptTemplate
# from langchain_community.llms import OpenAI

# # This controls how each document will be formatted. Specifically,
# # it will be passed to `format_document` - see that function for more
# # details.
# document_prompt = PromptTemplate(
#     input_variables=["page_content"],
#      template="{page_content}"
# )
# document_variable_name = "context"
# llm = model
# # The prompt here should take as an input variable the
# # `document_variable_name`
# prompt = PromptTemplate.from_template(
#     "Summarize this content: {context}"
# )
# llm_chain = LLMChain(llm=llm, prompt=prompt)
# # We now define how to combine these summaries
# reduce_prompt = PromptTemplate.from_template(
#     "Combine these summaries: {context}"
# )
# reduce_llm_chain = LLMChain(llm=llm, prompt=reduce_prompt)
# combine_documents_chain = StuffDocumentsChain(
#     llm_chain=reduce_llm_chain,
#     document_prompt=document_prompt,
#     document_variable_name=document_variable_name
# )
# reduce_documents_chain = ReduceDocumentsChain(
#     combine_documents_chain=combine_documents_chain,
# )
# chain = MapReduceDocumentsChain(
#     llm_chain=llm_chain,
#     reduce_documents_chain=reduce_documents_chain,
# )
# # If we wanted to, we could also pass in collapse_documents_chain
# # which is specifically aimed at collapsing documents BEFORE
# # the final call.
# prompt = PromptTemplate.from_template(
#     "Collapse this content: {context}"
# )
# llm_chain = LLMChain(llm=llm, prompt=prompt)
# collapse_documents_chain = StuffDocumentsChain(
#     llm_chain=llm_chain,
#     document_prompt=document_prompt,
#     document_variable_name=document_variable_name
# )
# reduce_documents_chain = ReduceDocumentsChain(
#     combine_documents_chain=combine_documents_chain,
#     collapse_documents_chain=collapse_documents_chain,
# )
# chain = MapReduceDocumentsChain(
#     llm_chain=llm_chain,
#     reduce_documents_chain=reduce_documents_chain,
# )

In [18]:
# chain.run(docs[:5])